In [11]:
import scipy.optimize as sco
import pandas as pd 
import numpy as np 
from numpy import array
from numpy import mean
from numpy import cov
from numpy.linalg import eig
from tqdm import tqdm
from pulp import *
import plotly.express as px 
from sklearn.decomposition import PCA
import random
import plotly.io as pio 
pio.renderers.default = "browser"
import plotly.graph_objects as go
from pandas_datareader import data

In [12]:
    def meanRetAn(data):             
        Result = 1
        
        for i in data:
            Result *= (1+i)
            
        Result = Result**(1/float(len(data)/52))-1
        
        return(Result)

In [44]:
df = pd.read_csv("ETFs_info.csv", index_col="Ticker")
df["Inception"] = pd.to_datetime(df["Inception"])
df = df[df["Inception"] <= "2015-01-01"]


fundamental_df = pd.read_csv("all_clusters.csv").rename(columns={"Unnamed: 0":"Ticker"}).set_index("Ticker") #Fundamental Clustering Data
weekly_return = pd.read_csv("WeeklyReturns.csv",index_col="Date") #Weekly Return Data
#fundamental_df = fundamental_df.loc[fundamental_df.index.intersection(weekly_return.columns)]
#fundamental_df = fundamental_df.loc[fundamental_df.index.intersection(df.index)]
len(fundamental_df)

695

In [46]:
fundamental_df.head()

,Category,Fundamental Cluster,Risk Cluster,kmean_env_cluster,Kmean_Social_cluster,Kmean_Gov_cluster
Ticker,,,,,,
SPY,Large Blend,High Fundamental Broad Market,Moderat/High Risk - Slightly Above Market Vola...,High Env_Responsibility,Low Social_Responsibility,High Gov_Responsibility
IVV,Large Blend,High Fundamental Broad Market,Moderat/High Risk - Slightly Above Market Vola...,High Env_Responsibility,Low Social_Responsibility,High Gov_Responsibility
VTI,Large Blend,High Fundamental Broad Market,Moderat/High Risk - Slightly Above Market Vola...,High Env_Responsibility,Low Social_Responsibility,High Gov_Responsibility
VOO,Large Blend,High Fundamental Broad Market,Moderat/High Risk - Slightly Above Market Vola...,High Env_Responsibility,Low Social_Responsibility,High Gov_Responsibility
QQQ,Large Growth,High Fundamental Broad Market,Moderat/High Risk - Slightly Above Market Vola...,High Env_Responsibility,Low Social_Responsibility,High Gov_Responsibility


In [47]:
ticks = list(fundamental_df.index)

In [314]:

# startDate = "2015-01-01"
# endDate = "2021-01-01"
# # DOWNLOAD THE DATA FROM YAHOO DATABASE
# #------------------------------------------------------------------
# dailyPrices = data.DataReader(ticks, 'yahoo', startDate, endDate)
# dailyPrices = dailyPrices["Adj Close"]

In [315]:
# dailyPrices.to_csv("Dailyprices.csv", index="Date")
# daily_prices = pd.read_csv("DailyPrices.csv", index_col="Date")

# PDI

In [66]:
def calculate_pdi( weekly_returns, portfolios, return_mean_range): 

    def meanRetAn(data):             
        Result = 1
        
        for i in data:
            Result *= (1+i)
            
        Result = Result**(1/float(len(data)/return_mean_range))-1
        
        return(Result)

    pca = PCA()
    PDI_dict = {}

    for i,y in tqdm(zip(portfolios,range(1,len(portfolios)+1))):
        n_assets = len(i)
        portfolio_weights_ew = np.repeat(1/n_assets, n_assets)
        port_ret  = weekly_returns[i].mul(portfolio_weights_ew,axis=1).sum(axis=1)

        ann_ret = meanRetAn(list(port_ret))
        an_cov = weekly_returns[i].cov()
        port_std = np.sqrt(np.dot(portfolio_weights_ew.T, np.dot(an_cov, portfolio_weights_ew)))*np.sqrt(return_mean_range)
        corr_matrix = np.array(weekly_returns[i].corr().replace(np.nan, 0))
        principalComponents = pca.fit(corr_matrix)
        PDI = 2*sum(principalComponents.explained_variance_ratio_*range(1,len(principalComponents.explained_variance_ratio_)+1,1))-1

        PDI_dict[y] = {}
        PDI_dict[y]["PDI_INDEX"] = PDI
        PDI_dict[y]["# of Assets"] = len(i)
        PDI_dict[y]["Assets"] = i
        PDI_dict[y]["Sharpe Ratio"] = ann_ret/port_std
        PDI_dict[y]["Annual Return"] = ann_ret
        PDI_dict[y]["Annual STD"] = port_std



    

        


    PDI_DF = pd.DataFrame(PDI_dict).T
    #PDI_DF["Assets"] = PDI_DF["Assets"].astype(str)
    PDI_DF["# of Assets"] = PDI_DF["# of Assets"].astype(str)
    PDI_DF["Sharpe Ratio"] = PDI_DF["Sharpe Ratio"].astype(float)
    PDI_DF["Annual STD"] = PDI_DF["Annual STD"].astype(float)
    PDI_DF["PDI_INDEX"] = PDI_DF["PDI_INDEX"].astype(float)
    PDI_DF["Annual Return"] = PDI_DF["Annual Return"].astype(float)
    SPY_DF = PDI_DF.iloc[0,:]
    return PDI_DF, SPY_DF #uncomment to add spy

# PDI

In [7]:
pdi_etf_start = pd.read_csv("WeeklyReturns.csv", index_col="Date").dropna(axis=1)
pdi_etf_crypto = pd.read_csv("crypto_weekly.csv", index_col="Date").dropna(axis=1)

In [352]:
random.seed(60)
num_pdi_ticker_etf = pdi_etf_start.columns
samples_etf = []
for number in range(2,10,1):
    for i in range(1,15000):
        #samples.extend([list(x) for x in combinations(selected_tickers, number_of_assets)])
        samples_etf.append(random.sample(list(num_pdi_ticker_etf),number))
# samples_mini_etf = []
# for i in samples_etf:
#     if i not in samples_mini_etf:
#         samples_mini_etf.append(i)

seen = set()
sampled_etf = [x for x in samples_etf if frozenset(x) not in seen and not seen.add(frozenset(x))]

print("Number of Portfolios ETF: {}".format(len(sampled_etf)))

Number of Portfolios ETF: 119510


In [354]:
pdi_df = calculate_pdi( weekly_returns = pdi_etf_start, portfolios = sampled_etf, return_mean_range = 52)

119510it [05:44, 346.97it/s]


In [362]:
fig = px.scatter(pdi_df, x = "PDI_INDEX", y = "Annual STD", color="# of Assets", hover_data=["Assets","Annual Return","Sharpe Ratio"])
fig.show()

In [363]:
fig = px.scatter(pdi_df, x = "PDI_INDEX", y = "Sharpe Ratio", color="# of Assets",hover_data=["Assets","Annual Return","Annual STD"])
fig.show()

In [364]:
fig = px.scatter(pdi_df, x = "PDI_INDEX", y = "Annual Return", color="# of Assets",hover_data=["Assets","Annual STD","Sharpe Ratio"])
fig.show()

## PDI Trading Strategy

In [67]:
def pca_per(return_data, initial_port, interval, ret_range_mean):
    data = return_data.copy() # data containing weekly returns
    tickers = list(data.columns)
    data.index = pd.to_datetime(data.index) # Conveting the index which is date to datetime
    weeks_list = data.index # grabbing all index dates
    data.index = data.index.to_period(interval) # converting the index to quarterly sets
    periods = data.index.unique() # taking the unique quarters to loop

    
    print(periods)
    first_period = periods[0] # the first period of the time frame
    remaining_periods = periods[1:] # the remianing periods of the time framr
    first_periods = periods[:-1] # all periods minus the last

    def meanRetAn(data):             
        Result = 1
        
        for i in data:
            Result *= (1+i)
            
        Result = Result**(1/float(len(data)/ret_range_mean))-1
        
        return(Result)

    
    def port_ret(returns,port, period): # function for calculating returns
        n_assets = len(port)
        portfolio_weights_ew = np.repeat(1/n_assets, n_assets)
        port_return = returns.loc[period][port].mul(portfolio_weights_ew,axis=1).sum(axis=1)
        return  port_return

    performance_eq = []
    performance_eq_pdi = []
    sharpe_performance_eq_2 = []
    ret_performance_eq_2 = []
    sharpe_performance_eq_2_pdi = []
    ret_performance_eq_2_pdi = []
    sharpe_performance_eq_3 = []
    ret_performance_eq_3 = []
    sharpe_performance_eq_3_pdi = []
    ret_performance_eq_3_pdi = []
    sharpe_performance_eq_max = []
    ret_performance_eq_max = []
    sharpe_performance_eq_max_pdi =  []
    ret_performance_eq_max_pdi =  []
    
    ############################################################ Calculate first period ####################################################################################
    #### Equal weight #####
    print(first_period)
    performance_eq.extend(port_ret(data, initial_port,first_period))
    sharpe_performance_eq_2.extend(port_ret(data, initial_port,first_period))
    sharpe_performance_eq_3.extend(port_ret(data, initial_port,first_period))
    sharpe_performance_eq_max.extend(port_ret(data, initial_port,first_period))
    tickers_weekly = list(data.columns)
    number_of_assets = [len(initial_port)]







    ######################################################## Calculation of portfolio perfomnce #############################################################################

    for init_time, next_time in zip(first_periods,remaining_periods):
        ############ Portfolio Creation ##############################
        samples = [["URTH"]]
        for number in [len(initial_port)]:
            for i in range(1,20000):
                #samples.extend([list(x) for x in combinations(selected_tickers, number_of_assets)])
                samples.append(random.sample(list(tickers),number))

        seen = set()
        samples_mini = [x for x in samples if frozenset(x) not in seen and not seen.add(frozenset(x))]

        print("Number of Portfolios: {}".format(len(samples_mini)))
        print("first time: {} - last time: {}".format(init_time, next_time))
        PDI_DF, SPY_DF = calculate_pdi(weekly_returns = data.loc[init_time].dropna(axis=1), portfolios = samples_mini, return_mean_range = ret_range_mean)


        ################################################## Taking the higest PDI ##########################################################################

        id = PDI_DF["PDI_INDEX"].idxmax()
        port_max_pdi = PDI_DF["Assets"][id]
        performance_eq_pdi.append(PDI_DF["PDI_INDEX"][id])
        
        port_max_ret_period = port_ret(data,port_max_pdi, next_time)
        performance_eq.extend(port_max_ret_period)

        ################################################## Taking the higest Sharpe Ration - PDI > 2##########################################################################
        id_sharpe = PDI_DF[PDI_DF["PDI_INDEX"]>2]["Annual Return"].idxmax()
        

        port_max_sharpe = PDI_DF["Assets"][id_sharpe]
        sharpe_performance_eq_2_pdi.append(PDI_DF["PDI_INDEX"][id_sharpe])


        
        port_max_ret_period_sharpe = port_ret(data,port_max_sharpe, next_time)
        sharpe_performance_eq_2.extend(port_max_ret_period_sharpe)

                
        ################################################## Taking the higest Sharpe Ration - PDI > 2##########################################################################
        try: 
            try:
                id_sharpe_3= PDI_DF[PDI_DF["PDI_INDEX"]>3]["Annual Return"].idxmax()
                port_max_sharpe_3 = PDI_DF["Assets"][id_sharpe_3]
                sharpe_performance_eq_3_pdi.append(PDI_DF["PDI_INDEX"][id_sharpe_3])
            except:
                id_sharpe_3= PDI_DF[PDI_DF["PDI_INDEX"]>2.5]["Annual Return"].idxmax()
                port_max_sharpe_3 = PDI_DF["Assets"][id_sharpe_3]
                sharpe_performance_eq_3_pdi.append(PDI_DF["PDI_INDEX"][id_sharpe_3])
        except:
            id_sharpe_3= PDI_DF["PDI_INDEX"].idxmax()
            port_max_sharpe_3 = PDI_DF["Assets"][id_sharpe_3]
            sharpe_performance_eq_3_pdi.append(PDI_DF["PDI_INDEX"][id_sharpe_3])


        
        port_max_ret_period_sharpe_3 = port_ret(data,port_max_sharpe_3, next_time)
        sharpe_performance_eq_3.extend(port_max_ret_period_sharpe_3)

        

        ################################################## Taking the higest Sharpe Ration - PDI > 2##########################################################################
        id_sharpe_max = PDI_DF["Annual Return"].idxmax()
        port_max_sharpe_max = PDI_DF["Assets"][id_sharpe_max]
        sharpe_performance_eq_max_pdi.append(PDI_DF["PDI_INDEX"][id_sharpe_max])
        port_max_ret_period_sharpe_max = port_ret(data,port_max_sharpe_max, next_time)
        sharpe_performance_eq_max.extend(port_max_ret_period_sharpe_max)




    dd = pd.DataFrame()
    dd["Time"] = weeks_list
    dd["per_pdi"] = performance_eq
    dd["per_sharpe_2"] = sharpe_performance_eq_2
    dd["per_sharpe_3"] = sharpe_performance_eq_3
    dd["per_sharpe_max"] = sharpe_performance_eq_max
    dd["URTH"] = list(data["URTH"])
    dd["per_pdi_cum"] = dd["per_pdi"].cumsum(axis=0)
    dd["per_sharpe_cum_2"] = dd["per_sharpe_2"].cumsum(axis=0)
    dd["per_sharpe_cum_3"] = dd["per_sharpe_3"].cumsum(axis=0)
    dd["per_sharpe_cum_max"] = dd["per_sharpe_max"].cumsum(axis=0)
    dd["URTH_cum"] = dd["URTH"].cumsum(axis=0)

    pdi = [performance_eq_pdi,sharpe_performance_eq_2_pdi,sharpe_performance_eq_3_pdi,sharpe_performance_eq_max_pdi]


    return dd,pdi

In [70]:
test = list(fundamental_df.loc[(fundamental_df['Risk Cluster']=="Moderat/High Risk - Slightly Above Market Volatility") & (fundamental_df['Fundamental Cluster']=="High Fundamental Broad Market") & (fundamental_df.kmean_env_cluster == "High Env_Responsibility") & (fundamental_df.Kmean_Social_cluster == "High Social_Responsibility")].index)
test.append("URTH")
week_return = pd.read_csv("WeeklyReturns.csv",index_col="Date") #Weekly Return Data
week_return_sort = week_return[test] #a

,["DAX", "XLK", "EMIF", "EMFM", "SOXL","SPY"],
    ["DAX", "XLK", "EMIF", "EMFM", "SOXL","FNDA", "QQXT"],["FNDA", "QQXT", "SPEU", "AIA", "XSMO","DAX", "XLK", "EMIF"]

In [71]:
ini_port = [list(week_return_sort.columns[:4]),list(week_return_sort.columns[:5])]
dd = {}
for i in ini_port:
    dd[len(i)]  = pca_per(return_data = week_return_sort, initial_port = i, interval = "Q", ret_range_mean = 4)
    

PeriodIndex(['2015Q1', '2015Q2', '2015Q3', '2015Q4', '2016Q1', '2016Q2',
             '2016Q3', '2016Q4', '2017Q1', '2017Q2', '2017Q3', '2017Q4',
             '2018Q1', '2018Q2', '2018Q3', '2018Q4', '2019Q1', '2019Q2',
             '2019Q3', '2019Q4', '2020Q1', '2020Q2', '2020Q3', '2020Q4'],
            dtype='period[Q-DEC]', name='Date', freq='Q-DEC')
2015Q1
0it [00:00, ?it/s]C:\Users\ALG\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\decomposition\_pca.py:468: RuntimeWarning:

invalid value encountered in true_divide

11it [00:00, 109.49it/s]Number of Portfolios: 19827
first time: 2015Q1 - last time: 2015Q2
19827it [02:02, 162.22it/s]
0it [00:00, ?it/s]C:\Users\ALG\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\decomposition\_pca.py:468: RuntimeWarning:

invalid value encountered in true_divide

27it [00:00, 262.84it/s]Number of Portfolios: 19828
first time: 2015Q2 - last time: 2015Q3
19828it [01:23, 237.90it/s]
0it [00:00, ?it/s]C:\Users\ALG\AppDa

In [72]:
pdi_4 = pd.DataFrame(dd[7][1]).T
pdi_4.columns = ["performance_eq_pdi","sharpe_performance_eq_2_pdi","sharpe_performance_eq_3_pdi","sharpe_performance_eq_max_pdi"]

KeyError: 7

In [479]:
fig = px.line(pdi_4, y=pdi_4.columns)
fig.show()

In [73]:
dd[4][0]

,Time,per_pdi,per_sharpe_2,per_sharpe_3,per_sharpe_max,URTH,per_pdi_cum,per_sharpe_cum_2,per_sharpe_cum_3,per_sharpe_cum_max,URTH_cum
0,2015-01-14,0.001588,0.001588,0.001588,0.001588,0.002868,0.001588,0.001588,0.001588,0.001588,0.002868
1,2015-01-21,0.008167,0.008167,0.008167,0.008167,0.009723,0.009755,0.009755,0.009755,0.009755,0.012590
2,2015-01-28,0.001787,0.001787,0.001787,0.001787,0.001416,0.011543,0.011543,0.011543,0.011543,0.014006
3,2015-02-04,0.009812,0.009812,0.009812,0.009812,0.007070,0.021354,0.021354,0.021354,0.021354,0.021076
4,2015-02-11,0.015284,0.015284,0.015284,0.015284,0.010812,0.036638,0.036638,0.036638,0.036638,0.031888
...,...,...,...,...,...,...,...,...,...,...,...
303,2020-12-02,-0.000056,0.002020,0.004623,-0.009444,0.010635,1.120348,1.587036,1.376258,1.371002,0.657226
304,2020-12-09,0.002555,0.016043,0.005470,0.011565,0.004082,1.122902,1.603079,1.381728,1.382568,0.661309
305,2020-12-16,0.062691,0.072328,0.064660,0.080509,0.011244,1.185593,1.675408,1.446388,1.463076,0.672552
306,2020-12-23,0.054035,0.082114,0.060688,0.104994,0.000090,1.239629,1.757522,1.507076,1.568070,0.672642


In [74]:
for i in dd.keys():
    fig = px.line(dd[i][0], x="Time", y = ["per_pdi_cum","URTH_cum","per_sharpe_cum_3","per_sharpe_cum_2","per_sharpe_cum_max"], \
                    title="Number of Assets: {} - period interval {}".format(i, "Q"))
    fig.show()

In [75]:
for i in dd.keys():
    print("Number of assets: {}".format(i))
    print(dd[i][0][["per_pdi","URTH","per_sharpe_3","per_sharpe_2","per_sharpe_max"]].std()*np.sqrt(252))
    print(50*"-")

Number of assets: 4
per_pdi           0.485184
URTH              0.359381
per_sharpe_3      0.523993
per_sharpe_2      0.575070
per_sharpe_max    0.604925
dtype: float64
--------------------------------------------------
Number of assets: 5
per_pdi           0.436987
URTH              0.359381
per_sharpe_3      0.502128
per_sharpe_2      0.540384
per_sharpe_max    0.573348
dtype: float64
--------------------------------------------------


In [442]:
fig = px.line(port_per_num, x="Time", y = ["per_pdi_cum","URTH_cum","per_sharpe_cum_3","per_sharpe_cum_2","per_sharpe_cum_max"], title="Number of Assets: {} - period interval {}".format(i, "Q"))
fig.show()

In [341]:
def pca_per(return_data, initial_port, interval, ret_range_mean):

    data = return_data.copy() # data containing weekly returns
    tickers = list(data.columns)
    data.index = pd.to_datetime(data.index) # Conveting the index which is date to datetime
    weeks_list = data.index # grabbing all index dates
    data.index = data.index.to_period(interval) # converting the index to quarterly sets
    periods = data.index.unique() # taking the unique quarters to loop

    
    print(periods)
    first_period = periods[0] # the first period of the time frame
    remaining_periods = periods[1:] # the remianing periods of the time framr
    first_periods = periods[:-1] # all periods minus the last

    def meanRetAn(data):             
        Result = 1
        
        for i in data:
            Result *= (1+i)
            
        Result = Result**(1/float(len(data)/ret_range_mean))-1
        
        return(Result)



    def optimal_weights(returns, port, lambda_l, range_period):


        def range_return_std(returns, port, weights):
            port_geo_ret = meanRetAn(list(returns[port].mul(weights,axis=1).sum(axis=1)))
            an_cov = returns[port].cov()
            port_std = np.sqrt(np.dot(weights.T, np.dot(an_cov, weights)))*np.sqrt(range_period)
            return  port_std,port_geo_ret




        def markowitz(weights, returns,lambda_l, port):
            p_var, p_ret = range_return_std(returns, port, weights)
            return -( (1- lambda_l)*p_ret - lambda_l*p_var )
            
        def markowitz_opt(returns, port,lambda_l):
            num_assets = len(port)
            args = (returns, lambda_l,port)
            constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
            bound = (0.1,1)
            bounds = tuple(bound for asset in range(num_assets))
            result = sco.minimize(markowitz, num_assets*[1./num_assets,], args=args,
                                method='SLSQP', bounds=bounds, constraints=constraints)
            return result



        mark = markowitz_opt(returns, port, lambda_l).x



        return list(mark)



    def port_ret(returns,port, period): # function for calculating returns
        n_assets = len(port)
        portfolio_weights_ew = np.repeat(1/n_assets, n_assets)
        port_return = returns.loc[period][port].mul(portfolio_weights_ew,axis=1).sum(axis=1)
        return  port_return


    
    def port_ret_w(returns,port,period,weights): # Calculating weightes returns
        port_return = returns.loc[period][port].mul(weights,axis=1).sum(axis=1)
        return  port_return



    performance_eq = []
    sharpe_performance_eq = []
    performance_w = []
    sharpe_performance_w = []
    weights = []
    weight_sr = []
    ############################################################ Calculate first period ####################################################################################
    #### Equal weight #####
    print(first_period)
    performance_eq.extend(port_ret(data, initial_port,first_period))
    sharpe_performance_eq.extend(port_ret(data, initial_port,first_period))
    tickers_weekly = list(data.columns)
    number_of_assets = [len(initial_port)]

    #### Weighted weight #####
    init_data = data.loc[first_period]
    ww = optimal_weights(init_data,initial_port,0.5,30)
    weights.append(ww)
    weight_sr.append(ww)
    performance_w.extend(port_ret_w(data, initial_port, first_period,ww))
    sharpe_performance_w.extend(port_ret_w(data, initial_port, first_period,ww))





    ################################################################## Portfolio Creation ###################################################################################
    samples = [["SPY"]]
    for number in [len(initial_port)]:
        for i in range(1,2000):
            #samples.extend([list(x) for x in combinations(selected_tickers, number_of_assets)])
            samples.append(random.sample(list(tickers),number))
    samples_mini = []
    for i in samples:
        if i not in samples_mini:
            samples_mini.append(i)

    print("Number of Portfolios: {}".format(len(samples_mini)))

    vol_d = {}
    ######################################################## Calculation of portfolio perfomnce #############################################################################

    for init_time, next_time in zip(first_periods,remaining_periods):
        print("first time: {} - last time: {}".format(init_time, next_time))
        PDI_DF, SPY_DF = calculate_pdi(weekly_returns = data.loc[init_time].dropna(axis=1), portfolios = samples_mini, return_mean_range = 30)

        q1,q2 = PDI_DF["PDI_INDEX"].quantile([0.20,0.85])
        r1,r2 = PDI_DF["Annual Return"].quantile([0.4,0.85])


        #Taking the higest return within the range
        SPY_vol_lower = SPY_DF["Annual STD"] - 0.04 # allowing for a little room of divergence to find a asset match
        SPY_vol_upper = SPY_DF["Annual STD"] + 0.01# allowing for a little room of divergence to find a asset match

        PDI_rev = PDI_DF[(PDI_DF["Annual STD"] >= SPY_vol_lower) & (PDI_DF["Annual STD"] <= SPY_vol_upper) & (PDI_DF["PDI_INDEX"] >= q1) & (PDI_DF["PDI_INDEX"] <= q2)\
                         & (PDI_DF["Annual STD"] <= SPY_vol_upper) & (PDI_DF["Annual Return"] >= r1) & (PDI_DF["Annual Return"] <= r2)\
                         & (PDI_DF["PDI_INDEX"] >= q1)].reset_index().drop(columns="index")


        # fig = px.scatter(PDI_DF,  color="Annual STD", y="Annual Return", x="PDI_INDEX")
        # fig.show()


        ################################################## Taking the higest return within the range ##########################################################################

        id = PDI_rev["Annual Return"].idxmax()
        port_max_ret = PDI_rev["Assets"][id]
        
        port_max_ret_period = port_ret(data,port_max_ret, next_time)
        performance_eq.extend(port_max_ret_period)

        ###### Weighted ###########
        period_return = data.loc[next_time]
        max_ww = optimal_weights(period_return,port_max_ret,0.5,30)
        weights.append(max_ww)
        port_max_ret_period_w = port_ret_w(data, port_max_ret,next_time,max_ww)
        performance_w.extend(port_max_ret_period_w)
    

        ################################################### Taking the best Sharpe Ratio within the range ######################################################################

        id_sharpe = PDI_rev["Sharpe Ratio"].idxmax()
        port_max_sharpe = PDI_rev["Assets"][id_sharpe]
        
        port_max_sharpe_period = port_ret(data,port_max_sharpe, next_time)
        sharpe_performance_eq.extend(port_max_sharpe_period)

        ###### Weighted ###########
        max_ww_sr = optimal_weights(period_return,port_max_sharpe,0.5,30)
        weight_sr.append(max_ww_sr)
        port_max_sharpe_period_w = port_ret_w(data, port_max_sharpe,next_time,max_ww_sr)
        sharpe_performance_w.extend(port_max_sharpe_period_w)






    dd = pd.DataFrame()
    dd["Time"] = weeks_list
    dd["per_eq"] = performance_eq
    dd["per_w"] = performance_w
    dd["SPY"] = list(data["SPY"])
    dd["sharpe_eq"] = sharpe_performance_eq
    dd["sharpe_w"] = sharpe_performance_w
    dd["per_eq_cum"] = dd["per_eq"].cumsum(axis=0)
    dd["per_w_cum"] = dd["per_w"].cumsum(axis=0)
    dd["SPY_cum"] = dd["SPY"].cumsum(axis=0)
    dd["sharpe_eq_cum"] = dd["sharpe_eq"].cumsum(axis=0)
    dd["sharpe_w_cum"] = dd["sharpe_w"].cumsum(axis=0)
    #fig = px.line(dd, x="Time", y = ["per_eq_cum","SPY_cum","sharpe_eq_cum","sharpe_w_cum","per_w_cum"], title="Number of Assets: {} - period interval {}".format(len(initial_port), interval))




    return fig, dd, weights,weight_sr




In [342]:
daily_prices = pd.read_csv("DailyPrices.csv", index_col="Date")

In [343]:
daily_prices = daily_prices.dropna(axis=1)

In [344]:
daily_prices = daily_prices.pct_change().dropna(axis=0)
daily_prices.head()

,SPY,IVV,VTI,VOO,QQQ,VEA,IEFA,VWO,VUG,IEMG,...,IDMO,PXJ,FLM,FTLB,NFTY,FTAG,CHII,CHIM,CHIE,LTL
Date,,,,,,,,,,,,,,,,,,,,,
2015-01-05,-0.018059,-0.017559,-0.017183,-0.017569,-0.014669,-0.022287,-0.022459,-0.015925,-0.017458,-0.015239,...,0.000000,-0.041714,-0.025698,0.004931,0.003053,-0.013078,-0.014325,0.073320,0.000746,-0.032846
2015-01-06,-0.009419,-0.009010,-0.009798,-0.009833,-0.013408,-0.011126,-0.010376,-0.004880,-0.009470,-0.005231,...,-0.048465,-0.022063,-0.011823,-0.020609,-0.017156,0.013252,-0.020069,-0.012018,0.000000,-0.025528
2015-01-07,0.012461,0.012371,0.011933,0.012495,0.012891,0.010703,0.010298,0.022974,0.012517,0.020596,...,0.008554,-0.006707,0.003682,0.008016,0.010417,0.011066,0.012712,0.003841,-0.020880,0.001067
2015-01-08,0.017745,0.017864,0.017640,0.017569,0.019140,0.015205,0.012787,0.016906,0.017619,0.017819,...,0.000000,0.014733,0.013984,0.000000,0.027584,0.015921,-0.002789,0.003189,0.044174,0.023091
2015-01-09,-0.008013,-0.008438,-0.008290,-0.008421,-0.006583,-0.004814,-0.004575,-0.004963,-0.007461,-0.003586,...,-0.002699,-0.017544,-0.009722,0.009941,-0.008134,0.036239,-0.005594,0.000000,-0.015317,-0.013194


In [345]:
ti = fundamental_df[(-fundamental_df["Category"].isin(['Trading--Leveraged Equity','Trading--Inverse Equity']))]
tttt = list(ti.index)
tttt.append("SPY")

daily_prices = daily_prices[daily_prices.columns.intersection(tttt)]

In [346]:
fig, dd, weights,weight_sr = pca_per(daily_prices,list(daily_prices.iloc[:,:4].columns), interval="Q",ret_range_mean=30)
fig.show()

0it [00:00, ?it/s]PeriodIndex(['2015Q1', '2015Q2', '2015Q3', '2015Q4', '2016Q1', '2016Q2',
             '2016Q3', '2016Q4', '2017Q1', '2017Q2', '2017Q3', '2017Q4',
             '2018Q1', '2018Q2', '2018Q3', '2018Q4', '2019Q1', '2019Q2',
             '2019Q3', '2019Q4', '2020Q1', '2020Q2', '2020Q3', '2020Q4'],
            dtype='period[Q-DEC]', name='Date', freq='Q-DEC')
2015Q1
Number of Portfolios: 2000
first time: 2015Q1 - last time: 2015Q2
2000it [00:05, 370.28it/s]


ValueError: too many values to unpack (expected 2)

In [70]:
dd

,Time,per_eq,per_w,SPY,sharpe_eq,sharpe_w,per_eq_cum,per_w_cum,SPY_cum,sharpe_eq_cum,sharpe_w_cum
0,2015-01-05,-0.019683,-0.021296,-0.018059,-0.019683,-0.021296,-0.019683,-0.021296,-0.018059,-0.019683,-0.021296
1,2015-01-06,-0.008950,-0.010037,-0.009419,-0.008950,-0.010037,-0.028633,-0.031333,-0.027478,-0.028633,-0.031333
2,2015-01-07,0.014109,0.011947,0.012461,0.014109,0.011947,-0.014524,-0.019386,-0.015017,-0.014524,-0.019386
3,2015-01-08,0.015661,0.014682,0.017745,0.015661,0.014682,0.001136,-0.004704,0.002728,0.001136,-0.004704
4,2015-01-09,-0.005591,-0.005055,-0.008013,-0.005591,-0.005055,-0.004455,-0.009759,-0.005286,-0.004455,-0.009759
...,...,...,...,...,...,...,...,...,...,...,...
1505,2020-12-24,0.002210,0.001451,0.003890,-0.001312,-0.004147,0.368491,0.877582,0.804173,0.553772,1.078221
1506,2020-12-28,-0.004679,-0.008660,0.008591,0.006064,0.005777,0.363813,0.868922,0.812764,0.559836,1.083998
1507,2020-12-29,-0.001510,-0.001861,-0.001908,0.005496,0.010330,0.362303,0.867062,0.810856,0.565332,1.094327
1508,2020-12-30,0.009130,0.012354,0.001427,0.004235,0.006554,0.371433,0.879415,0.812283,0.569567,1.100881


In [68]:
#dddd = PDI_DF[PDI_DF["# of Assets"] == "14" ]
dddd = dd.copy()
#dddd = PDI_DF[PDI_DF["Annual STD"] <= 0.2 ]

In [482]:
fig = px.bar(dddd, color="# of Assets", y="Annual Return", x="PDI_INDEX")
fig.show()

In [483]:
fig = px.scatter(dddd,  color="# of Assets", y="Annual Return", x="PDI_INDEX")
fig.show()

In [486]:
fig = px.box(dddd, x="# of Assets", y="PDI_INDEX")
fig.show()

In [498]:
fig = px.scatter_3d(dddd, x='STD SEGG', y = "Annual Return", z = "PDI_INDEX", color="# of Assets", hover_data = [dddd.index], title = "ETF Portfolios")
fig.show()

In [462]:
fig = px.scatter(dddd, x="PDI_INDEX", y="Annual Return", color = "STD SEG")
fig.show()

## PDI of total universe througout time 

In [106]:
time_pdi_etf = pd.read_csv("WeeklyReturns.csv", index_col="Date").dropna(axis=1)
time_pdi_crypto = pd.read_csv("crypto_weekly.csv", index_col="Date").dropna(axis=1)
etf_volme = pd.read_csv("ETF_Volumne.csv", index_col="Date").drop(columns="Unnamed: 0")
crypto_volme = pd.read_csv("Crypto_Volumne.csv", index_col="Date").drop(columns="Unnamed: 0")

In [107]:
# startDate = "2015-01-01"
# endDate = "2021-01-01"
# etf_names = list(time_pdi_crypto.columns)
# dailyPrices = data.DataReader(etf_names, 'yahoo', startDate, endDate)
# dailyPrices 
# dailyPrices["Volume"].reset_index(drop=False).to_csv("Crypto_Volumne.csv")

In [108]:
#ETF volume config
etf_volme.index = pd.to_datetime(etf_volme.index)
etf_volme.index = etf_volme.index.to_period("Q")
#Crypto volume config
crypto_volme.index = pd.to_datetime(crypto_volme.index)
crypto_volme.index = crypto_volme.index.to_period("Q")

#etf
groupie_volume_etf = etf_volme.groupby("Date").sum()
groupie_volume_etf = groupie_volume_etf.groupby("Date").sum()
volume_dic_etf = dict(zip(list(groupie_volume_etf.sum(axis=1).index.astype(str)),groupie_volume_etf.sum(axis=1)))
#crypto
groupie_volume_crypto = crypto_volme.groupby("Date").sum()
groupie_volume_crypto = groupie_volume_crypto.groupby("Date").sum()
volume_dic_crypto = dict(zip(list(groupie_volume_crypto.sum(axis=1).index.astype(str)),groupie_volume_crypto.sum(axis=1)))

In [88]:
time_pdi_etf.index = pd.to_datetime(time_pdi_etf.index)
time_pdi_crypto.index = pd.to_datetime(time_pdi_crypto.index)
time_pdi_etf.index = time_pdi_etf.index.to_period("Q")
time_pdi_crypto.index = time_pdi_crypto.index.to_period("Q")

In [63]:
def pdi_q(x):    
    corr_matrix = np.array(x.corr().replace(np.nan, 0))
    principalComponents = pca.fit(corr_matrix)
    PDI = 2*sum(principalComponents.explained_variance_ratio_*range(1,len(principalComponents.explained_variance_ratio_)+1,1))-1
    return PDI


In [109]:
time_pdi_etf.index = pd.to_datetime(time_pdi_etf.index)
time_pdi_crypto.index = pd.to_datetime(time_pdi_crypto.index)
time_pdi_etf.index = time_pdi_etf.index.to_period("Q")
time_pdi_crypto.index = time_pdi_crypto.index.to_period("Q")
#Time Series periods
quarters = list(time_pdi_etf.index.unique())

etf_pdi = {}
crypto_pdi = {}
pca = PCA()
for q in quarters:
    etf_pdi[str(q)] = pdi_q(time_pdi_etf.loc[q])
    crypto_pdi[str(q)] = pdi_q(time_pdi_crypto.loc[q])

In [65]:
fig = px.line(x=list(etf_pdi.keys()), y=list(etf_pdi.values()))
fig.show()

In [66]:
fig = px.line(x=list(crypto_pdi.keys()), y=list(crypto_pdi.values()))
fig.show()

In [163]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(rows=1, cols=2,
                    specs=[[{"secondary_y": True}, {"secondary_y": True}]],subplot_titles=("ETF Universe", "Crypto Universe"),horizontal_spacing = 0.05)



# Add traces
fig.add_trace(
    go.Scatter(x=list(volume_dic_etf.keys()), y=list(volume_dic_etf.values()),line=dict(color='mediumseagreen', width=2), name="ETF - Volume"),
    secondary_y=False,
    row=1, col=1
)

fig.add_trace(
    go.Scatter(x=list(etf_pdi.keys()), y=list(etf_pdi.values()),line=dict(color='mediumslateblue', width=2), name="ETF - PDI"),
    secondary_y=True,
    row=1, col=1
)


# Add traces
fig.add_trace(
    go.Scatter(x=list(volume_dic_crypto.keys()), y=list(volume_dic_crypto.values()),line=dict(color='steelblue', width=2) ,name="Crypto - Volume"),
    secondary_y=False, row=1, col=2
)

fig.add_trace(
    go.Scatter(x=list(crypto_pdi.keys()), y=list(crypto_pdi.values()),line=dict(color='darkslateblue', width=2), name="Crypto - PDI",text=["Crypto - PDI"]),
    secondary_y=True, row=1, col=2
)
# Set x-axis title
fig.update_xaxes(title_text="Quarterly Periods", row=1,col=1, tickangle = 90)
# Set x-axis title
fig.update_xaxes(title_text="Quarterly Periods", row=1,col=2, tickangle = 90)
# Set y-axes titles
fig.update_yaxes(title_text="Volume", secondary_y=False, row=1,col=1)
#fig.update_yaxes(title_text="Diversification Index - ETF Universe", secondary_y=True,row=1,col=1)

#fig.update_yaxes(title_text="Volume - Crypto Universe", secondary_y=False, row=1,col=2)
fig.update_yaxes(title_text="Diversification Index", secondary_y=True,row=1,col=2)


fig.update_layout(height=600,showlegend=True,template="plotly",legend=dict(
    #yanchor="top",
    y=1,
    #xanchor="left",
    x=0.97
))
fig.show()

## Number of Assets VS PDI

In [224]:
num_vs_pdi_etf = pd.read_csv("WeeklyReturns.csv", index_col="Date").dropna(axis=1)
num_vs_pdi_crypto = pd.read_csv("crypto_weekly.csv", index_col="Date").dropna(axis=1)

In [225]:
random.seed(60)
num_pdi_ticker_etf = num_vs_pdi_etf.columns
samples_etf = []
for number in range(2,30,1):
    for i in range(1,7000):
        #samples.extend([list(x) for x in combinations(selected_tickers, number_of_assets)])
        samples_etf.append(random.sample(list(num_pdi_ticker_etf),number))
# samples_mini_etf = []
# for i in samples_etf:
#     if i not in samples_mini_etf:
#         samples_mini_etf.append(i)

seen = set()
sampled_etf = [x for x in samples_etf if frozenset(x) not in seen and not seen.add(frozenset(x))]

print("Number of Portfolios ETF: {}".format(len(sampled_etf)))
print(60 *"-")

num_pdi_ticker_crypto = num_vs_pdi_crypto.columns
samples_crypto = []
for number in range(2,len(num_pdi_ticker_crypto)+1,1):
    for i in range(1,7000):
        #samples.extend([list(x) for x in combinations(selected_tickers, number_of_assets)])
        samples_crypto.append(random.sample(list(num_pdi_ticker_crypto),number))
# samples_mini_crypto = []
# for i in samples_crypto:
#     if i not in samples_mini_crypto:
#         samples_mini_crypto.append(i)

seen = set()
sampled_crypto = [x for x in samples_crypto if frozenset(x) not in seen and not seen.add(frozenset(x))]

print("Number of Portfolios crypto: {}".format(len(sampled_crypto)))

Number of Portfolios ETF: 195868
------------------------------------------------------------
Number of Portfolios crypto: 502


In [227]:
num_PDI_PDI_DF_etf = calculate_pdi( weekly_returns = num_vs_pdi_etf, portfolios = sampled_etf, return_mean_range = 52)
num_PDI_PDI_DF_crypto = calculate_pdi( weekly_returns = num_vs_pdi_crypto, portfolios = sampled_crypto, return_mean_range = 52)

195868it [10:45, 303.35it/s]
502it [00:01, 353.63it/s]


In [229]:
num_PDI_PDI_DF_etf["str_PDI"] = num_PDI_PDI_DF_etf["PDI_INDEX"].round(0).astype(int)
num_PDI_PDI_DF_crypto["str_PDI"] = num_PDI_PDI_DF_crypto["PDI_INDEX"].round(0).astype(int)

In [230]:
pio.templates.default = "simple_white"
px.defaults.template = "plotly"
px.defaults.color_continuous_scale = px.colors.sequential.Blackbody

In [239]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = go.Figure()
# Initialize figure with subplots
fig = make_subplots(
    rows=1, cols=2, subplot_titles=("Portfolio Diversification Index - ETF Universe", "Portfolio Diversification Index - Crypto Universe"))

# Add traces
fig.add_trace(go.Box(x=num_PDI_PDI_DF_etf["# of Assets"], y=num_PDI_PDI_DF_etf["PDI_INDEX"]),row=1, col=1)
fig.add_trace(go.Box(x=num_PDI_PDI_DF_crypto["# of Assets"], y=num_PDI_PDI_DF_crypto["PDI_INDEX"]), row=1, col=2)


# Update xaxis properties
fig.update_xaxes(title_text="Number of Assets in Portfolio", row=1, col=1)
fig.update_xaxes(title_text="Number of Assets in Portfolio", row=1, col=2)

# Update yaxis properties
fig.update_yaxes(title_text="Diversification Index", row=1, col=1)
fig.update_yaxes(title_text="Diversification Index",row=1, col=2)

fig.update_layout(height=1100,showlegend=False,template="plotly")

fig.show()

In [94]:
 marker=dict(color='steelblue'))
,marker=dict(color='darkslateblue')

In [50]:
fig = px.box(num_PDI_PDI_DF_etf, x="str_PDI", y="Sharpe Ratio", color="# of Assets")
fig.update_layout(width = 1200, height=700)
fig.update_xaxes(title_text = "Number of Unique Investments in the Portfolios")
fig.update_yaxes(title_text = "Risk Adjusted Return")
fig.show()

In [18]:
fig = px.box(num_PDI_PDI_DF_crypto, x="str_PDI", y="Sharpe Ratio", color="# of Assets")
fig.update_layout(width = 1200, height=700)
fig.update_xaxes(title_text = "Number of Unique Investments in the Portfolios")
fig.update_yaxes(title_text = "Risk Adjusted Return")
fig.show()


In [148]:
# ETF
dict = {}
for i in num_PDI_PDI_DF_etf["str_PDI"].unique():
    dict[i] = {}
    for x in num_PDI_PDI_DF_etf["# of Assets"].unique():
        dict[i][x] = num_PDI_PDI_DF_etf[(num_PDI_PDI_DF_etf["str_PDI"]==i) & (num_PDI_PDI_DF_etf["# of Assets"]==x)]["Sharpe Ratio"].skew()
skew = pd.DataFrame(dict)
skew_df = pd.DataFrame(skew.stack()).reset_index()
skew_df.rename(columns=({"level_0":"num_assets","level_1":"pdi", 0:"skewness" }), inplace=True)

In [ ]:
fig = px.bar(skew_df,x="pdi", y="skewness", color="num_assets", barmode="group")
fig.update_yaxes(title="Skewness of distribution")
fig.update_xaxes(title="Portfolio Diversification")
fig.update_layout(height = 700, width= 1200, legend_title = " # of Assets")
fig.show()

In [151]:
# Crypto
dict_crypto = {}
for i in num_PDI_PDI_DF_crypto["str_PDI"].unique():
    dict_crypto[i] = {}
    for x in num_PDI_PDI_DF_crypto["# of Assets"].unique():
        dict_crypto[i][x] = num_PDI_PDI_DF_crypto[(num_PDI_PDI_DF_crypto["str_PDI"]==i) & (num_PDI_PDI_DF_crypto["# of Assets"]==x)]["Sharpe Ratio"].skew()
skew = pd.DataFrame(dict_crypto)
skew_df_crypto = pd.DataFrame(skew.stack()).reset_index()
skew_df_crypto.rename(columns=({"level_0":"num_assets","level_1":"pdi", 0:"skewness" }), inplace=True)

In [152]:
fig = px.bar(skew_df_crypto,x="pdi", y="skewness", color="num_assets", barmode="group")
fig.update_yaxes(title="Skewness of distribution")
fig.update_xaxes(title="Portfolio Diversification")
fig.update_layout(height = 700, width= 1200, legend_title = " # of Assets")
fig.show()

#plotting color 

aliceblue, antiquewhite, aqua, aquamarine, azure,
            beige, bisque, black, blanchedalmond, blue,
            blueviolet, brown, burlywood, cadetblue,
            chartreuse, chocolate, coral, cornflowerblue,
            cornsilk, crimson, cyan, darkblue, darkcyan,
            darkgoldenrod, darkgray, darkgrey, darkgreen,
            darkkhaki, darkmagenta, darkolivegreen, darkorange,
            darkorchid, darkred, darksalmon, darkseagreen,
            darkslateblue, darkslategray, darkslategrey,
            darkturquoise, darkviolet, deeppink, deepskyblue,
            dimgray, dimgrey, dodgerblue, firebrick,
            floralwhite, forestgreen, fuchsia, gainsboro,
            ghostwhite, gold, goldenrod, gray, grey, green,
            greenyellow, honeydew, hotpink, indianred, indigo,
            ivory, khaki, lavender, lavenderblush, lawngreen,
            lemonchiffon, lightblue, lightcoral, lightcyan,
            lightgoldenrodyellow, lightgray, lightgrey,
            lightgreen, lightpink, lightsalmon, lightseagreen,
            lightskyblue, lightslategray, lightslategrey,
            lightsteelblue, lightyellow, lime, limegreen,
            linen, magenta, maroon, mediumaquamarine,
            mediumblue, mediumorchid, mediumpurple,
            mediumseagreen, mediumslateblue, mediumspringgreen,
            mediumturquoise, mediumvioletred, midnightblue,
            mintcream, mistyrose, moccasin, navajowhite, navy,
            oldlace, olive, olivedrab, orange, orangered,
            orchid, palegoldenrod, palegreen, paleturquoise,
            palevioletred, papayawhip, peachpuff, peru, pink,
            plum, powderblue, purple, red, rosybrown,
            royalblue, rebeccapurple, saddlebrown, salmon,
            sandybrown, seagreen, seashell, sienna, silver,
            skyblue, slateblue, slategray, slategrey, snow,
            springgreen, steelblue, tan, teal, thistle, tomato,
            turquoise, violet, wheat, white, whitesmoke,
            yellow, yellowgreen

In [291]:
# Initialize figure with subplots
import plotly.graph_objects as go
from plotly.subplots import make_subplots
fig = go.Figure()
fig = make_subplots(
    rows=1, cols=2, subplot_titles=("Correlation Heatmap - ETF Universe", "Correlation Heatmap- Crypto Universe")
)


fig.add_trace(go.Heatmap(z=num_vs_pdi_etf.corr(),
                    x=num_vs_pdi_etf.corr().columns,
                    y=num_vs_pdi_etf.corr().index,colorbar=dict(y=.5,x=0.45,len=1),colorscale='cividis'), row=1, col=1)

fig.add_trace(go.Heatmap(z=num_vs_pdi_crypto.corr(),
                    x=num_vs_pdi_crypto.corr().columns,
                    y=num_vs_pdi_crypto.corr().index,colorbar=dict(y=.5,x=1,len=1),colorscale='cividis'), row=1, col=2)


fig.update_layout(height=600,showlegend=False,template="plotly")

fig.show()

## Catogries in PDI

In [256]:
random.seed(60)
num_pdi_ticker_etf = num_vs_pdi_etf.columns
samples_etf_min = []
for number in range(2,10,1):
    for i in range(1,20000):
        #samples.extend([list(x) for x in combinations(selected_tickers, number_of_assets)])
        samples_etf_min.append(random.sample(list(num_pdi_ticker_etf),number))
# samples_mini_etf = []
# for i in samples_etf:
#     if i not in samples_mini_etf:
#         samples_mini_etf.append(i)

seen = set()
sampled_etf_min = [x for x in samples_etf_min if frozenset(x) not in seen and not seen.add(frozenset(x))]

print("Number of Portfolios ETF: {}".format(len(sampled_etf_min)))
print(60 *"-")

Number of Portfolios ETF: 159160
------------------------------------------------------------


In [257]:
def category_ratio(x):
    return len(category.loc[x]["Category"].unique())/len(category.loc[x])

category = pd.DataFrame(fundamental_df["Category"])


In [258]:
num_PDI_PDI_DF_etf_min = calculate_pdi( weekly_returns = num_vs_pdi_etf, portfolios = sampled_etf_min, return_mean_range = 52)


159160it [07:31, 352.21it/s]


In [262]:
num_PDI_PDI_DF_etf_min["category_ratio"] = num_PDI_PDI_DF_etf_min["Assets"].apply(lambda x: category_ratio(x))

In [366]:
num_PDI_PDI_DF_etf_min[num_PDI_PDI_DF_etf_min["# of Assets"] == 5]

,PDI_INDEX,# of Assets,Assets,Sharpe Ratio,Annual Return,Annual STD,category_ratio,str_PDI
59166,1.986036,5,"[ARGT, HEEM, RYJ, VDC, IXP]",0.524288,0.091450,0.174427,1.0,2
59167,1.094982,5,"[GDXJ, RYJ, DGRO, IWF, PKW]",0.711353,0.141482,0.198892,0.8,1
59168,1.645919,5,"[DAX, XLK, EMIF, EMFM, SOXL]",0.695012,0.202056,0.290723,0.8,2
59169,1.447085,5,"[VEU, DVYA, PJP, PKW, EURL]",0.271667,0.066068,0.243197,1.0,1
59170,1.291155,5,"[PFM, IWN, HEWG, NGE, SPXL]",0.335852,0.078848,0.234770,0.8,1
...,...,...,...,...,...,...,...,...
79160,1.718449,5,"[FNDA, QQXT, SPEU, AIA, XSMO]",0.587468,0.111439,0.189693,1.0,2
79161,1.794645,5,"[CFA, RPG, PID, TUR, WBIE]",0.283557,0.048595,0.171376,0.8,2
79162,1.764517,5,"[RZV, PIN, GVAL, UXI, QQQE]",0.454891,0.103337,0.227170,1.0,2
79163,2.466436,5,"[XLI, EIS, TOK, AIRR, FENY]",0.314014,0.067537,0.215076,0.8,2


In [268]:
num_PDI_PDI_DF_etf_min["# of Assets"] = num_PDI_PDI_DF_etf_min["# of Assets"].astype(int)
num_PDI_PDI_DF_etf_min["str_PDI"] = num_PDI_PDI_DF_etf_min["PDI_INDEX"].round(0).astype(int)

In [308]:
fig = px.box(num_PDI_PDI_DF_etf, x="str_PDI", y="category_ratio", color = "# of Assets")
fig.update_layout(width = 1200, height=700)
fig.update_xaxes(title_text = "Portfolio Diversification")
fig.update_yaxes(title_text = "Category Ratio")
fig.show()

In [281]:
import plotly.figure_factory as ff

#### Portfolios containing both crypto and ETF's

In [102]:
num_vs_pdi_etf_1 = pd.read_csv("WeeklyReturns.csv", index_col="Date").dropna(axis=1)
num_vs_pdi_crypto_1 = pd.read_csv("crypto_weekly.csv", index_col="Date").dropna(axis=1)

comb_df_num_pdi_merge = pd.merge(num_vs_pdi_etf_1,num_vs_pdi_crypto_1, right_index=True,left_index=True)

In [103]:
num_pdi_ticker_etf = num_vs_pdi_etf_1.columns
num_pdi_ticker_crypto = num_vs_pdi_crypto_1.columns
#50% crypto
samples_etf_50 = []
for number in range(1,9,1):
    for i in range(1,10000):
        sampled_etf = []
        sampled_crypto = []
        sampled_etf = random.sample(list(num_pdi_ticker_etf),number)
        sampled_crypto = random.sample(list(num_pdi_ticker_crypto),number)
        samples_etf_50.append(sampled_etf+sampled_crypto)

# samples_mini_etf = []
# for i in samples_etf:
#     if i not in samples_mini_etf:
#         samples_mini_etf.append(i)
seen = set()
samples_combined_unique_50 = [x for x in samples_etf_50 if frozenset(x) not in seen and not seen.add(frozenset(x))]
print("Number of Portfolios 50%: {}".format(len(samples_combined_unique_50)))

#25% crypto
samples_etf_25 = []
for number in range(1,9,1):
    for i in range(1,10000):
        sampled_etf = []
        sampled_crypto = []
        sampled_etf = random.sample(list(num_pdi_ticker_etf),number*3)
        sampled_crypto = random.sample(list(num_pdi_ticker_crypto),number)
        samples_etf_25.append(sampled_etf+sampled_crypto)

# samples_mini_etf_1 = []
# for i in samples_etf_1:
#     if i not in samples_mini_etf_1:
#         samples_mini_etf_1.append(i)
seen = set()
samples_combined_unique_25  = [x for x in samples_etf_25 if frozenset(x) not in seen and not seen.add(frozenset(x))]
print("Number of Portfolios 25%: {}".format(len(samples_combined_unique_25)))

#75% crypto
samples_etf_75 = []
for number in [1,2,3]:
    for i in range(1,10000):
        sampled_etf = []
        sampled_crypto = []
        sampled_etf = random.sample(list(num_pdi_ticker_etf),number)
        sampled_crypto = random.sample(list(num_pdi_ticker_crypto),number*3)
        samples_etf_75.append(sampled_etf+sampled_crypto)

# samples_mini_etf_1 = []
# for i in samples_etf_1:
#     if i not in samples_mini_etf_1:
#         samples_mini_etf_1.append(i)
samples_combined_unique_75  = [x for x in samples_etf_75 if frozenset(x) not in seen and not seen.add(frozenset(x))]
print("Number of Portfolios 75%: {}".format(len(samples_combined_unique_75)))

Number of Portfolios 50%: 74971
Number of Portfolios 25%: 79991
Number of Portfolios 75%: 29178


In [104]:
#25%
num_PDI_PDI_DF_combined_25 = calculate_pdi( weekly_returns = comb_df_num_pdi_merge, portfolios = samples_combined_unique_25, return_mean_range = 52) 
#50%
num_PDI_PDI_DF_combined_50 = calculate_pdi( weekly_returns = comb_df_num_pdi_merge, portfolios = samples_combined_unique_50, return_mean_range = 52)
#75%
num_PDI_PDI_DF_combined_75 = calculate_pdi( weekly_returns = comb_df_num_pdi_merge, portfolios = samples_combined_unique_75, return_mean_range = 52)


79991it [04:18, 309.84it/s]
74971it [03:32, 352.19it/s]
29178it [01:16, 380.25it/s]


In [105]:
#25%
num_PDI_PDI_DF_combined_25["pdi_round"] = num_PDI_PDI_DF_combined_25["PDI_INDEX"].round(0)
#50%
num_PDI_PDI_DF_combined_50["pdi_round"] = num_PDI_PDI_DF_combined_50["PDI_INDEX"].round(0)
#75%
num_PDI_PDI_DF_combined_75["pdi_round"] = num_PDI_PDI_DF_combined_75["PDI_INDEX"].round(0)


In [106]:
num_PDI_PDI_DF_combined_25

,PDI_INDEX,# of Assets,Assets,Sharpe Ratio,Annual Return,Annual STD,pdi_round
1,1.811927,4,"[GOEX, RDIV, IXN, XMR-USD]",1.683537,0.670852,0.398478,2.0
2,1.093890,4,"[FTC, VIDI, RWL, DOGE-USD]",1.039024,0.431621,0.415410,1.0
3,1.207313,4,"[SLYG, SPY, PJP, BTC-USD]",1.531157,0.374506,0.244590,1.0
4,1.546015,4,"[CSM, IDU, ITA, DOGE-USD]",1.094292,0.445522,0.407133,2.0
5,1.814487,4,"[GLIN, XSLV, HACK, NMC-USD]",0.479115,0.191561,0.399824,2.0
...,...,...,...,...,...,...,...
79987,1.773431,32,"[GII, FTHI, IDMO, GRES, VUG, IPO, SQQQ, MMTM, ...",1.767501,0.619608,0.350556,2.0
79988,2.281716,32,"[PSCD, RWJ, GII, QAT, FNK, INDA, DWM, GLIN, FD...",1.788682,0.648300,0.362446,2.0
79989,2.611525,32,"[FEP, FEM, KWEB, IXG, IPOS, XBI, EWSC, SILJ, F...",1.523749,0.486983,0.319596,3.0
79990,1.883128,32,"[KIE, ARKW, UWM, PZD, UDOW, XSLV, RSP, AMLP, R...",1.745711,0.662940,0.379754,2.0


In [107]:
# 25% crypto
fig = px.box(num_PDI_PDI_DF_combined_25,x="pdi_round",y="Sharpe Ratio", color="# of Assets")
fig.update_layout(height=500,showlegend=True,template="plotly", width = 1000)
fig.update_xaxes(title_text="Portfolio Diversification Index")
fig.show()

# 50% crypto
fig = px.box(num_PDI_PDI_DF_combined_50,x="pdi_round",y="Sharpe Ratio", color="# of Assets")
fig.update_layout(height=500,showlegend=True,template="plotly", width = 1000)
fig.update_xaxes(title_text="Portfolio Diversification Index")
fig.show()

# 75% crypto
fig = px.box(num_PDI_PDI_DF_combined_75,x="pdi_round",y="Sharpe Ratio", color="# of Assets")
fig.update_layout(height=500,showlegend=True,template="plotly", width = 1000)
fig.update_xaxes(title_text="Portfolio Diversification Index")
fig.show()


In [257]:

# 75% crypto
fig = px.box(num_PDI_PDI_DF_combined_50,x="# of Assets",y="PDI_INDEX")
fig.update_layout(height=500,showlegend=True,template="plotly", width = 1000)
fig.update_xaxes(title_text="Portfolio Diversification Index")
fig.show()


In [182]:
num_PDI_PDI_DF_combined_1["pdi_round"] = num_PDI_PDI_DF_combined_1["PDI_INDEX"].round(0)
num_PDI_PDI_DF_combined_1

,PDI_INDEX,# of Assets,Assets,Sharpe Ratio,Annual Return,Annual STD,pdi_round
1,1.634053,4,"[WBIL, RYT, LTC-USD, DASH-USD]",1.517404,0.901926,0.594388,2.0
2,1.481993,4,"[IWM, ARKW, DOGE-USD, LTC-USD]",1.460219,0.948599,0.649628,1.0
3,1.424294,4,"[ARKQ, PSCF, LTC-USD, XRP-USD]",1.221026,0.861077,0.705208,1.0
4,1.654358,4,"[PSCU, IWS, DASH-USD, XRP-USD]",1.358502,0.856951,0.630806,2.0
5,1.606256,4,"[SH, CFA, DASH-USD, XMR-USD]",1.615124,0.942331,0.583442,2.0
...,...,...,...,...,...,...,...
69956,2.319995,16,"[SPEU, EWQ, PKW, DOO, ARKQ, FBZ, PEJ, HEWG, XM...",2.064649,1.300381,0.629831,2.0
69957,2.540826,16,"[VOT, PKW, EXI, CHIE, BBH, FIDU, IPKW, XLK, DO...",1.770113,0.924223,0.522126,3.0
69958,2.686009,16,"[IWF, QWLD, GDXJ, IJK, SPEU, DWM, FPXI, PHO, N...",2.047691,1.349373,0.658973,3.0
69959,2.141026,16,"[IDHQ, EWY, FEUZ, MXI, ILF, TZA, VIG, HAP, LTC...",2.068912,1.315505,0.635844,2.0


In [183]:
fig = px.box(num_PDI_PDI_DF_combined_1, x="pdi_round", y="Sharpe Ratio", color="# of Assets")
fig.show()

In [ ]:
#1/4 Ratio - 20% Crypto

In [158]:
num_pdi_ticker_etf = num_vs_pdi_etf_1.columns
num_pdi_ticker_crypto = num_vs_pdi_crypto_1.columns
samples_etf_1 = []
for number in range(1,9,1):
    for i in range(1,7000):
        sampled_etf = []
        sampled_crypto = []
        sampled_etf = random.sample(list(num_pdi_ticker_etf),number*4)
        sampled_crypto = random.sample(list(num_pdi_ticker_crypto),number)
        samples_etf_1.append(sampled_etf+sampled_crypto)

samples_mini_etf_1 = []
for i in samples_etf_1:
    if i not in samples_mini_etf_1:
        samples_mini_etf_1.append(i)
samples_combined_unique_1  = [x for x in samples_mini_etf_1 if frozenset(x) not in seen and not seen.add(frozenset(x))]
print("Number of Portfolios Combined: {}".format(len(samples_combined_unique_1)))

Number of Portfolios Combined: 55992


In [159]:
num_PDI_PDI_DF_combined_2 = calculate_pdi( weekly_returns = comb_df_num_pdi_merge, portfolios = samples_combined_unique_1, return_mean_range = 52)


55992it [03:10, 293.50it/s]


In [108]:
# Initialize figure with subplots
import plotly.graph_objects as go
from plotly.subplots import make_subplots
fig = go.Figure()
fig = make_subplots(
    rows=1, cols=1
)

# Add traces
fig.add_trace(go.Box(num_PDI_PDI_DF_combined_2,x="# of Assets", y="PDI_INDEX",marker=dict(color='steelblue')), row=1, col=1)

# Update xaxis properties
#fig.update_xaxes(title_text="Percentage of cryptos in the portfolio's", row=1, col=1)


# Update yaxis properties
fig.update_yaxes(title_text="Portfolio Diversification", row=1, col=1)


fig.update_layout(height=600,showlegend=False,template="plotly", width = 1100)

fig.show()

NameError: name 'num_PDI_PDI_DF_combined_2' is not defined

In [161]:
fig = px.scatter(num_PDI_PDI_DF_combined_2, x="PDI_INDEX", y="Sharpe Ratio", color="# of Assets")
fig.show()

In [163]:
# Initialize figure with subplots
import plotly.graph_objects as go
from plotly.subplots import make_subplots
fig = go.Figure()
fig = make_subplots(
    rows=1, cols=1
)

# Add traces
fig.add_trace(go.Box(x=num_PDI_PDI_DF_combined_2["PDI_INDEX"], y=num_PDI_PDI_DF_combined_2["Annual STD"],marker=dict(color='steelblue')), row=1, col=1)

# Update xaxis properties
#fig.update_xaxes(title_text="Percentage of cryptos in the portfolio's", row=1, col=1)


# Update yaxis properties
fig.update_yaxes(title_text="Standard Deviation", row=1, col=1)


fig.update_layout(height=600,showlegend=False,template="plotly", width = 1100)

fig.show()

#### Portfolios containing both crypto and ETF's

In [5]:
samples = []
for number in [10,9,8,7,6,5,4,3,2,1]:
    for i in range(1,8000):
        #samples.extend([list(x) for x in combinations(selected_tickers, number_of_assets)])
        samples.append(random.sample(list(num_vs_pdi_etf.columns),number))


print("Number of Portfolios: {}".format(len(samples)))

Number of Portfolios: 79990


In [8]:
per_cryp = []
for i in samples:
    if int(len(i)) == 10:
        per = np.round(1-(int(len(i))/10),1)
        per_cryp.append(per)
    else:
        per = np.round(1-(int(len(i))/10),1)
        per_cryp.append(per)

In [12]:
new_port = []
for i in samples:
    if int(len(i)) == 10:
        new_port.append(i)
    else:
        new_port.append(i.extend(random.sample(list(num_vs_pdi_crypto.columns),10-len(i))))

print("Number of Portfolios: {}".format(len(new_port)))

Number of Portfolios: 79990


In [17]:
num_vs_pdi_combined = pd.merge(num_vs_pdi_etf,num_vs_pdi_crypto, left_index=True,right_index=True, how="inner")

In [18]:
num_PDI_PDI_DF_combined = calculate_pdi( weekly_returns = num_vs_pdi_combined, portfolios = new_port, return_mean_range = 52)

79990it [03:41, 360.53it/s]


In [19]:
num_PDI_PDI_DF_combined["Percentage_Crypto"] = per_cryp

In [20]:
num_PDI_PDI_DF_combined

,PDI_INDEX,# of Assets,Assets,Sharpe Ratio,Annual Return,Annual STD,Percentage_Crypto
1,2.724998,10,"[IEO, SPEU, SPUU, DDM, CFO, EWL, IVW, RZV, IEU...",0.444796,0.097024,0.218131,0.0
2,1.192735,10,"[DBEU, VTWO, SCHV, PEZ, SCHF, PSQ, DTH, PSCD, ...",0.378020,0.057570,0.152292,0.0
3,3.121719,10,"[EZM, GXC, IWM, CSF, ARKK, QQXT, XBI, DVYE, CA...",0.681106,0.130335,0.191359,0.0
4,2.538988,10,"[PSI, EEMO, GDXJ, PKB, IEV, ITA, EQWL, RPV, PX...",0.669173,0.135878,0.203053,0.0
5,2.134212,10,"[CFA, VAW, VB, EEMA, SLX, FVD, MOAT, PBS, EEMV...",0.599214,0.112083,0.187050,0.0
...,...,...,...,...,...,...,...
79986,4.820370,10,"[YINN, BTC-USD, NMC-USD, DOGE-USD, XRP-USD, XL...",2.169165,2.326518,1.072541,0.9
79987,4.989466,10,"[QQQE, XVG-USD, BTC-USD, XMR-USD, NMC-USD, DAS...",2.210480,2.356302,1.065969,0.9
79988,4.867727,10,"[ILF, NMC-USD, DOGE-USD, XMR-USD, DASH-USD, XR...",2.169409,2.315373,1.067283,0.9
79989,4.916087,10,"[FDL, DOGE-USD, XVG-USD, BTC-USD, XMR-USD, DAS...",2.188186,2.330589,1.065078,0.9


In [56]:
# Initialize figure with subplots
import plotly.graph_objects as go
from plotly.subplots import make_subplots
fig = go.Figure()
fig = make_subplots(
    rows=1, cols=1
)

# Add traces
fig.add_trace(go.Box(x=num_PDI_PDI_DF_combined["Percentage_Crypto"], y=num_PDI_PDI_DF_combined["PDI_INDEX"],marker=dict(color='steelblue')), row=1, col=1)

# Update xaxis properties
fig.update_xaxes(title_text="Percentage of cryptos in the portfolio's", row=1, col=1)


# Update yaxis properties
fig.update_yaxes(title_text="Diversification Index", row=1, col=1)


fig.update_layout(height=600,showlegend=False,template="plotly",title_text="Portfolio Diversificaiton Index - Percentage Allocated Cryptos", width = 1100)

fig.show()

## PDI vs Risk


In [111]:
num_PDI_PDI_DF_combined_25["allo_cryp"] = "25"
num_PDI_PDI_DF_combined_50["allo_cryp"] = "50"
num_PDI_PDI_DF_combined_75["allo_cryp"] = "75"

In [114]:
concat_all = pd.concat([num_PDI_PDI_DF_combined_25,num_PDI_PDI_DF_combined_50,num_PDI_PDI_DF_combined_75])

In [116]:
# 25% crypto
fig = px.box(concat_all,x="pdi_round",y="Annual STD", color="# of Assets")
fig.update_layout(height=500,showlegend=True,template="plotly", width = 1000)
fig.update_xaxes(title_text="Portfolio Diversification Index")
fig.show()



In [118]:
# 25% crypto
fig = px.box(num_PDI_PDI_DF_combined_25,x="pdi_round",y="Annual STD")
fig.update_layout(height=500,showlegend=True,template="plotly", width = 1000)
fig.update_xaxes(title_text="Portfolio Diversification Index")

fig.show()

# 50% crypto
fig = px.box(num_PDI_PDI_DF_combined_50,x="pdi_round",y="Annual STD")
fig.update_layout(height=500,showlegend=True,template="plotly", width = 1000)
fig.update_xaxes(title_text="Portfolio Diversification Index")

fig.show()

# 75% crypto
fig = px.box(num_PDI_PDI_DF_combined_75,x="pdi_round",y="Annual STD")
fig.update_layout(height=500,showlegend=True,template="plotly", width = 1000)
fig.update_xaxes(title_text="Portfolio Diversification Index")
fig.show()

In [7]:
num_PDI_PDI_DF_etf = calculate_pdi( weekly_returns = num_vs_pdi_etf, portfolios = samples_mini_etf, return_mean_range = 52)
#num_PDI_PDI_DF_crypto = calculate_pdi( weekly_returns = num_vs_pdi_crypto, portfolios = samples_mini_crypto, return_mean_range = 52)

59997it [02:49, 354.25it/s]


In [12]:
num_PDI_PDI_DF_etf["str_PDI"] = num_PDI_PDI_DF_etf["PDI_INDEX"].round(0)
num_PDI_PDI_DF_etf["str_PDI"] = num_PDI_PDI_DF_etf["str_PDI"].astype(int)
num_PDI_PDI_DF_etf["str_PDI"] = num_PDI_PDI_DF_etf["str_PDI"].astype(str)

# num_PDI_PDI_DF_crypto["str_PDI"] = num_PDI_PDI_DF_crypto["PDI_INDEX"].round(0)
# num_PDI_PDI_DF_crypto["str_PDI"] = num_PDI_PDI_DF_crypto["str_PDI"].astype(int)
# num_PDI_PDI_DF_crypto["str_PDI"] = num_PDI_PDI_DF_crypto["str_PDI"].astype(str)

In [60]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = go.Figure()
# Initialize figure with subplots
fig = make_subplots(
    rows=2, cols=2, subplot_titles=("ETF Universe", "Crypto Universe")
)

# Add traces
fig.add_trace(go.Box(x=num_PDI_PDI_DF_etf["str_PDI"], y=num_PDI_PDI_DF_etf["Annual STD"],marker=dict(color='steelblue')), row=1, col=1)
fig.add_trace(go.Box(x=num_PDI_PDI_DF_crypto["str_PDI"], y=num_PDI_PDI_DF_crypto["Annual STD"],marker=dict(color='darkslateblue')), row=1, col=2)


# Update xaxis properties
fig.update_xaxes(title_text="Portfolio Diversification Index", row=1, col=1)
fig.update_xaxes(title_text="Portfolio Diversification Index", row=1, col=2)

# Update yaxis properties
fig.update_yaxes(title_text="Volatility", row=1, col=1)
fig.update_yaxes(title_text="Volatility",row=1, col=2)

fig.update_layout(height=1100,showlegend=False,template="plotly")

fig.show()

In [59]:
fig = px.scatter(num_PDI_PDI_DF_crypto, x="PDI_INDEX", y="Annual STD", color="# of Assets")
fig.show()

In [40]:
list_1

{'1': 0.18191475554760222, '2': 0.19508316249051075, '3': 0.20171149762356633}

## Portoflio Diversification

In [39]:
pdi_div_etf = pd.read_csv("WeeklyReturns.csv", index_col="Date").dropna(axis=1)

In [40]:
num_pdi_ticker_etf = pdi_div_etf.columns
samples_etf = []
for number in range(2,17,1):
    for i in range(1,5000):
        #samples.extend([list(x) for x in combinations(selected_tickers, number_of_assets)])
        samples_etf.append(random.sample(list(num_pdi_ticker_etf),number))
samples_mini_etf = []
for i in samples_etf:
    if i not in samples_mini_etf:
        samples_mini_etf.append(i)

print("Number of Portfolios ETF: {}".format(len(samples_mini_etf)))
print(60 *"-")


Number of Portfolios ETF: 74963
------------------------------------------------------------


In [41]:
PDI_div_DF_etf = calculate_pdi( weekly_returns = pdi_div_etf, portfolios = samples_mini_etf, return_mean_range = 52)

74963it [03:52, 322.68it/s]


In [42]:
fundamental_df

,Category,Fundamental Cluster,Risk Cluster
SPY,Large Blend,High Fundamental Broad Market,Moderat/High Risk - Slightly Above Market Vola...
IVV,Large Blend,High Fundamental Broad Market,Moderat/High Risk - Slightly Above Market Vola...
VTI,Large Blend,High Fundamental Broad Market,Moderat/High Risk - Slightly Above Market Vola...
VOO,Large Blend,High Fundamental Broad Market,Moderat/High Risk - Slightly Above Market Vola...
QQQ,Large Growth,High Fundamental Broad Market,Moderat/High Risk - Slightly Above Market Vola...
...,...,...,...
FTAG,Natural Resources,Mid Fundamentals Broad Market,Moderat Risk - Slighty Below Market Volatility
CHII,China Region,Low Fundamentals Broad Market,Moderat Risk - Slighty Below Market Volatility
CHIM,China Region,Low Fundamentals Broad Market,Moderat/High Risk - Slightly Above Market Vola...
CHIE,China Region,Low Fundamentals Broad Market,Moderat Risk - Slighty Below Market Volatility


## Correlation Martrix VS Covariance 

In [73]:
from sklearn.decomposition import PCA
pca =PCA()

In [74]:
data_cor_cov = pd.read_csv("DailyPrices.csv", index_col="Date").dropna(axis=1)
num = 300
data_cor_cov = data_cor_cov.iloc[:,num:num+5].pct_change().dropna(axis=0)
data_cor_cov

,UDOW,EPI,IYK,PEY,FNX
Date,,,,,
2015-01-05,-0.052006,-0.012091,-0.011233,-0.015672,-0.018064
2015-01-06,-0.026049,-0.029918,-0.005729,-0.006823,-0.013362
2015-01-07,0.038547,0.017757,0.016894,0.011450,0.011580
2015-01-08,0.054383,0.028007,0.016230,0.013585,0.015910
2015-01-09,-0.026524,0.001340,-0.008127,-0.008190,-0.011268
...,...,...,...,...,...
2020-12-24,0.007941,0.008481,0.006830,0.002324,-0.001823
2020-12-28,0.019293,0.011212,0.002222,0.002899,-0.003819
2020-12-29,-0.006144,0.003119,-0.002801,-0.007514,-0.013059


### Correlation can only maintaine equal weights in calculations 


### Correlation 

In [75]:

principalComponents_cor = pca.fit(data_cor_cov.corr())
PDI_cor = 2*sum(principalComponents_cor.explained_variance_ratio_*range(1,len(principalComponents_cor.explained_variance_ratio_)+1,1))-1
print("Correlation PDI: {}".format(PDI_cor))

Correlation PDI: 1.4691751294366147


### Covariance 

#### Equal Weight

In [76]:
principalComponents_cov = pca.fit(data_cor_cov.cov())
PDI_cov = 2*sum(principalComponents_cov.explained_variance_ratio_*range(1,len(principalComponents_cov.explained_variance_ratio_)+1,1))-1
print("Equal weight Covariance PDI: {}".format(PDI_cov))

Equal weight Covariance PDI: 1.0337430425723007


### Weigthed 

In [77]:
weights = [ 0.2,0.2,0.2,0.2,0.2]
w_data = data_cor_cov.mul(weights)

principalComponents_cov_w = pca.fit(w_data.cov())
PDI_cov_w = 2*sum(principalComponents_cov_w.explained_variance_ratio_*range(1,len(principalComponents_cov_w.explained_variance_ratio_)+1,1))-1
print("Weight Covariance PDI: {}".format(PDI_cov_w))

Weight Covariance PDI: 1.0337430425723002


In [142]:
import random
n = 5
w = [[0.2,0.2,0.2,0.2,0.2]]
for i in range(1,100000):
    weights = [random.random() for _ in range(n)]
    sum_weights = sum(weights)
    weights = [1*w/sum_weights for w in weights]
    w.append(list(np.round(weights,2)))

In [143]:
def calculate_pdi_weights( returns,return_mean_range, weights): 

    def meanRetAn(data):             
        Result = 1
        
        for i in data:
            Result *= (1+i)
            
        Result = Result**(1/float(len(data)/return_mean_range))-1
        
        return(Result)

    pca = PCA()
    PDI_dict = {}

    for y in tqdm(weights):
        port_ret  = returns.mul(y,axis=1).sum(axis=1)

        ann_ret = meanRetAn(list(port_ret))
        an_cov = returns.cov()
        port_std = np.sqrt(np.dot(np.array(y).T, np.dot(an_cov, y)))*np.sqrt(return_mean_range)
        corr_matrix = np.array(returns.mul(y).cov())
        principalComponents = pca.fit(corr_matrix)
        PDI = 2*sum(principalComponents.explained_variance_ratio_*range(1,len(principalComponents.explained_variance_ratio_)+1,1))-1

        PDI_dict[str(y)] = {}
        PDI_dict[str(y)]["PDI_INDEX"] = PDI
        PDI_dict[str(y)]["# of Assets"] = len(y)
        PDI_dict[str(y)]["Sharpe Ratio"] = ann_ret/port_std
        PDI_dict[str(y)]["Annual Return"] = ann_ret
        PDI_dict[str(y)]["Annual STD"] = port_std

    return PDI_dict


In [149]:
data_cor_cov_1 = pd.read_csv("DailyPrices.csv", index_col="Date").dropna(axis=1)
assets = ["VLU","QTEC","LIT","IXP","BJK"]
data_cor_cov_1 = data_cor_cov_1[assets].pct_change().dropna(axis=0)
data_cor_cov_1

,VLU,QTEC,LIT,IXP,BJK
Date,,,,,
2015-01-05,0.000000,-0.014662,-0.021333,-0.019336,-0.023040
2015-01-06,-0.028224,-0.021855,0.000000,-0.003740,-0.018704
2015-01-07,0.015441,0.009983,0.000000,0.011090,0.017956
2015-01-08,0.000000,0.022594,0.011807,0.016369,0.007870
2015-01-09,0.010258,-0.000921,-0.000898,-0.002324,-0.003500
...,...,...,...,...,...
2020-12-24,-0.001331,0.005044,-0.008037,-0.003952,0.004064
2020-12-28,0.004999,-0.004497,0.026840,0.009850,-0.003835
2020-12-29,-0.004891,-0.004444,-0.025810,0.002167,-0.003208


In [150]:
aa = calculate_pdi_weights(data_cor_cov_1,252,w)

100%|██████████| 100000/100000 [03:08<00:00, 530.41it/s]


In [151]:
weight_df = pd.DataFrame(aa).T.reset_index()
weight_df.head()

,index,PDI_INDEX,# of Assets,Sharpe Ratio,Annual Return,Annual STD
0,"[0.2, 0.2, 0.2, 0.2, 0.2]",2.245102,5.0,0.710304,0.137860,0.194086
1,"[0.12, 0.27, 0.26, 0.33, 0.01]",1.205199,5.0,0.796164,0.153273,0.192514
2,"[0.19, 0.09, 0.51, 0.18, 0.03]",1.006471,5.0,0.777796,0.162808,0.209319
3,"[0.09, 0.3, 0.25, 0.19, 0.17]",1.252533,5.0,0.772163,0.156835,0.203111
4,"[0.21, 0.19, 0.19, 0.26, 0.15]",2.471552,5.0,0.712602,0.135098,0.189584


In [180]:
fig = px.scatter(weight_df, y="Annual STD", x="PDI_INDEX", hover_data=["index"], color="Annual Return", title = "Highest Combintation PDI - And then weighted")
fig.show()

In [155]:
fig = px.scatter_3d(weight_df, z="Annual Return", y="Annual STD", hover_data=["index"],color="Sharpe Ratio", x = "PDI_INDEX",title = "Highest Combintation PDI - And then weighted")
fig.show()

## Without crypto's 

In [106]:
etf = pd.read_csv("WeeklyReturns.csv", index_col= "Date")
etf_name = list(etf.columns)

In [108]:
samples_etf = []

for number in [5]:
    for i in range(1,100000):
        #samples.extend([list(x) for x in combinations(selected_tickers, number_of_assets)])
        samples_etf.append(random.sample(etf_name,number))
samples_mini = []
for i in samples_etf:
    if i not in samples_mini:
        samples_mini.append(i)

print("Number of Portfolios: {}".format(len(samples_mini)))






Number of Portfolios: 99999


In [110]:
len(samples_mini)

99999

In [117]:
PDI_DF, SPY_DF = calculate_pdi(etf, samples_mini, 52)

99999it [04:14, 392.21it/s]


In [119]:
PDI_DF

,PDI_INDEX,# of Assets,Assets,Sharpe Ratio,Annual Return,Annual STD
1,1.319746,5,"[CHIC, IGF, GWX, RWK, JKI]",0.378834,0.072948,0.192560
2,1.594580,5,"[CZA, SYG, XLV, ACWI, CHIE]",0.542822,0.086214,0.158825
3,1.471866,5,"[IGV, DIG, FILL, IPKW, VTI]",0.183430,0.050372,0.274614
4,2.125803,5,"[RSP, ISRA, SOXX, DDM, ICLN]",0.808880,0.184427,0.228003
5,1.583234,5,"[SLY, IHI, PSCE, SUSA, RZV]",0.182064,0.043980,0.241563
...,...,...,...,...,...,...
99995,1.960906,5,"[EXI, XRT, EWS, SDOG, HAP]",0.333575,0.063779,0.191197
99996,1.886686,5,"[RETL, EWN, PRN, SPYV, FPXI]",0.629290,0.162200,0.257750
99997,2.233961,5,"[IWR, DEEP, HACK, IVV, HEZU]",0.549769,0.104218,0.189567
99998,1.682608,5,"[XSVM, DDM, IEMG, FDL, VFH]",0.488911,0.108017,0.220933


In [128]:
fig = px.scatter_3d(PDI_DF, x ='PDI_INDEX', y= "Annual Return", hover_data=["Assets"], z = "Annual STD", color = "Sharpe Ratio")
fig.show()

In [ ]:
fig = px.scatter(PDI_DF, x ='PDI_INDEX',  color ="Annual STD", y= "Sharpe Ratio")
fig.show()

In [ ]:
fig = px.scatter(PDI_DF, x ='PDI_INDEX',  y="Annual STD")
fig.show()

In [129]:
PDI_DF["Quantile"] = pd.qcut(PDI_DF["PDI_INDEX"],4)

In [133]:
fig = px.scatter_3d(PDI_DF, x ='Annual STD',  color ="Quantile", y= "Annual Return", z="PDI_INDEX" )
fig.show()

## Weigthing cryptoes in different sizes of a portfolio

In [43]:
crypto = pd.read_csv("crypto_weekly.csv",index_col="Date")
crypto_name = list(crypto.columns)
etf = pd.read_csv("WeeklyReturns.csv", index_col= "Date")
etf_name = list(etf.columns)
comb_df = pd.merge(crypto,etf, right_index=True,left_index=True)

In [44]:
ti = list(comb_df.columns)

samples = []
for number in [10,9,8,7,6,5,4,3,2,1]:
    for i in range(1,3000):
        #samples.extend([list(x) for x in combinations(selected_tickers, number_of_assets)])
        samples.append(random.sample(etf_name,number))
samples_mini = []
for i in samples:
    if i not in samples_mini:
        samples_mini.append(i)

print("Number of Portfolios: {}".format(len(samples_mini)))

#PDI_DF, SPY_DF = calculate_pdi(weekly_return, samples_mini, return_mean_range = 52)



Number of Portfolios: 27664


In [45]:
samples_mini[2997:3002]

[['ZCAN', 'EFA', 'IWX', 'RPV', 'SDS', 'EUDG', 'LIT', 'ENFR', 'HEEM', 'SPLV'],
 ['VOT', 'TZA', 'NORW', 'PSCF', 'PRN', 'IXUS', 'VPU', 'SLX', 'EDOG', 'EWW'],
 ['EWSC', 'RTH', 'RZG', 'XTL', 'SOXS', 'VPU', 'JKG', 'MXI', 'FVD'],
 ['TECL', 'EWGS', 'ZCAN', 'LGLV', 'FJP', 'PHDG', 'EWY', 'VLU', 'RZG'],
 ['QEMM', 'INDY', 'FCG', 'IVE', 'FILL', 'VUSE', 'EFAV', 'EWSC', 'BKF']]

In [46]:
per_cryp = []
for i in samples_mini:
    if int(len(i)) == 10:
        per = np.round(1-(int(len(i))/10),1)
        per_cryp.append(per)
    else:
        per = np.round(1-(int(len(i))/10),1)
        per_cryp.append(per)

In [53]:

new_port = []
for i in samples_mini:
    if len(i) == 10:
        new_port.append(i)
    else:
        new_port.append(i.extend(random.sample(crypto_name,10-int(len(i)))))

print("Number of Portfolios: {}".format(len(new_port)))

Number of Portfolios: 27664


In [54]:
PDI_DF = calculate_pdi(comb_df, new_port, return_mean_range = 52)

27664it [01:17, 358.02it/s]


In [55]:
PDI_DF["per_crypto"] = per_cryp # adding the percentage of cryptoes in the portfolio
PDI_DF["per_crypto"] = PDI_DF["per_crypto"].astype(str)

In [75]:
fig = px.scatter(PDI_DF, x ='PDI_INDEX',hover_data=["Assets"],  color ="per_crypto", y= "Sharpe Ratio",color_discrete_sequence=px.colors.qualitative.Vivid, title = "Percentage of Crypto's allocated in Portfolios",opacity=0.8)
fig.update_layout(legend_title_text='Percentage of Crypto', height=700, width=1200)
fig.update_yaxes( title="Risk Adjusted Return")
fig.update_xaxes(title="Portfolio Diversification")
fig.show()

In [81]:
fig = px.scatter(PDI_DF, x ='Annual STD',hover_data=["Assets"],  color ="per_crypto", y= "Annual Return",color_discrete_sequence=px.colors.qualitative.Vivid, title = "Percentage of Crypto's allocated in Portfolios",opacity=0.8)
fig.update_layout(legend_title_text='Percentage of Crypto', height=700, width=1200)
fig.update_yaxes( title="Annual Return")
fig.update_xaxes(title="Annual Standard Deviation")
fig.show()

In [96]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = go.Figure()
# Initialize figure with subplots


# Add traces
fig.add_trace(go.Box(x=PDI_DF["per_crypto"], y=PDI_DF["Annual STD"],marker=dict(color='steelblue')))


# Update xaxis properties
fig.update_xaxes(title_text="Weight of Allocated Crypto")


# Update yaxis properties
fig.update_yaxes(title_text="Volatility - Annual Standard Deviation")

fig.update_layout(height=600,showlegend=False,template="plotly", width=1100)

fig.show()

## 